<a href="https://colab.research.google.com/github/SU-sumico/dlj/blob/main/3_cnn_cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

目的
* ネコとイヌの画像データと畳み込みニューラルネットワーク（CNN）を用いて分類を行う。

準備
* GPUを用いて処理を行うために、上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください。

使用するデータセット
* データセットは、RGBカラーの画像、ネコとイヌのクラスに分類されています。
https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset?resource=download

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator

img_w, img_h = 150, 150        # 画像をリサイズするときのサイズ
batch_size = 32                # ミニバッチのサイズ

# 訓練データを読み込んで加工するジェネレーターを生成
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,         # RGB値を0～1の範囲に変換
    rotation_range=15,         # 15度の範囲でランダムに回転させる
    zoom_range=0.2,            # ランダムに拡大
    horizontal_flip=True       # 水平方向にランダムに反転
    )

# テストデータを読み込むジェネレーターを生成
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# 訓練データをミニバッチの数だけ生成
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep learning/3data/train',               # 訓練データのフォルダー
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

# テストデータをミニバッチの数だけ生成
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Deep learning/3data/validation',          # テストデータのフォルダー
    target_size=(img_w, img_h), # 画像をリサイズ
    batch_size=batch_size,      # ミニバッチのサイズ
    class_mode='binary')        # 二値分類なのでbinaryを指定

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [3]:
# 正解ラベルを確認
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras import optimizers

# モデルを構築
model = Sequential()

# （第1層）畳み込み層
model.add(
    Conv2D(
        filters=32,                # フィルターの数は32
        kernel_size=(3, 3),        # 3×3のフィルターを使用
        input_shape=(150, 150, 3), # 入力データの形状
        padding='same',            # ゼロパディングを行う
        activation='relu'          # 活性化関数はReLU
        ))
# （第2層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第3層）畳み込み層
model.add(
    Conv2D(
        filters=32,            # フィルターの数は32
        kernel_size=(3, 3),    # 3×3のフィルターを使用
        activation='relu'      # 活性化関数はReLU
        ))
# （第4層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2))
)
# ドロップアウト25％
model.add(Dropout(0.25))

# （第5層）畳み込み層
model.add(
    Conv2D(filters=64,         # フィルターの数は64
           kernel_size=(3, 3), # 3×3のフィルターを使用
           activation='relu'   # 活性化関数はReLU
          ))
# （第6層）プーリング層
model.add(
    MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト25％
model.add(Dropout(0.25))

# 出力層への入力を4階テンソルから2階テンソルに変換する
model.add(Flatten())

# （第7層）全結合層
model.add(
    Dense(64,                  # ニューロン数は64
          activation='relu'))  # 活性化関数はReLU
# ドロップアウト50％
model.add(Dropout(0.5))

# （第8層）出力層
model.add(
    Dense(
        1,                     # ニューロン数は1個
        activation='sigmoid')) # 活性化関数はsigmoid

# モデルのコンパイル
model.compile(
    loss='binary_crossentropy', # バイナリ用の交差エントロピー誤差
    metrics=['accuracy'],       # 学習評価として正解率を指定
    # Adamアルゴリズムで最適化
    optimizer=optimizers.Adam(),
)

# モデルのサマリを表示
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 75, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 73, 73, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 36, 36, 32)        0

In [5]:
epochs = 60             # 学習回数
batch_size = batch_size # 設定済みのミニバッチのサイズ

# 学習を行う
history = model.fit_generator(
    train_generator,    # 訓練データ
    epochs=epochs,      # 学習回数
    verbose=1,          # 学習の進捗状況を出力する
    # テストデータ
    validation_data=validation_generator,
)

<ipython-input-5-fcec4d70183c>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/60
63/63 [==============================] - 666s 11s/step - loss: 0.7225 - accuracy: 0.4950 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 2/60
63/63 [==============================] - 22s 346ms/step - loss: 0.6919 - accuracy: 0.4930 - val_loss: 0.6904 - val_accuracy: 0.5500
Epoch 3/60
63/63 [==============================] - 19s 297ms/step - loss: 0.6866 - accuracy: 0.5240 - val_loss: 0.6706 - val_accuracy: 0.5325
Epoch 4/60
63/63 [==============================] - 20s 318ms/step - loss: 0.6891 - accuracy: 0.5365 - val_loss: 0.6896 - val_accuracy: 0.5362
Epoch 5/60
63/63 [==============================] - 19s 305ms/step - loss: 0.6642 - accuracy: 0.5570 - val_loss: 0.6551 - val_accuracy: 0.6037
Epoch 6/60
63/63 [==============================] - 20s 311ms/step - loss: 0.6562 - accuracy: 0.5935 - val_loss: 0.6466 - val_accuracy: 0.6200
Epoch 7/60
63/63 [==============================] - 20s 317ms/step - loss: 0.6505 - accuracy: 0.6070 - val_loss: 0.6458 - val_accuracy: 0.6438


テストデータによるネコとイヌの識別は、正解率が77.3%。データ拡張しても精度は低い。